$$
Entrenamiento
$$

In [1]:
# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise import dump
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#importar datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
df_movie = df_movie.reset_index(drop=True)

In [2]:
# visualizar peliculas
df_movie.head()

,id,title
0,ns1,dick johnson is dead
1,ns2,blood & water
2,ns3,ganglands
3,ns4,jailbirds new orleans
4,ns5,kota factory


In [3]:
# visualizar los ratings de peliculas por usuario
df_scores.head()

,userId,score,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


Documentación Entrenamiento de Modelo

## Código para entrenar y guardar un modelo SVD

El siguiente código utiliza la biblioteca Surprise para entrenar un modelo SVD (Singular Value Decomposition) en un conjunto de datos de películas y calificaciones. El modelo entrenado se guarda en un archivo pkl para su uso posterior.

### Paso 1: Importar las bibliotecas necesarias
Se importan las bibliotecas necesarias, incluyendo `dump` de Surprise, que se utiliza para guardar el modelo entrenado.

### Paso 2: Importar los datos
Se importan los datos de películas y calificaciones desde archivos CSV. El DataFrame `df_movie` contiene información sobre las películas, mientras que `df_scores` contiene las calificaciones de los usuarios para cada película.

### Paso 3: Crear un objeto Reader
Se crea un objeto Reader que se utilizará para analizar los datos de entrada. Se especifica la escala de calificación entre 1.0 y 5.0.

### Paso 4: Limitar los datos y crear el conjunto de datos
Para evitar el sobreajuste durante el entrenamiento, se limita la cantidad de datos a utilizar. Se carga el conjunto de datos a partir de las columnas relevantes en el DataFrame `df_scores` utilizando el objeto Reader creado anteriormente.

### Paso 5: Dividir los datos en conjuntos de entrenamiento y prueba
Se dividen los datos en conjuntos de entrenamiento y prueba utilizando la función `train_test_split` de Surprise. Se utiliza un tamaño de prueba del 30% y se fija una semilla aleatoria para reproducibilidad.

### Paso 6: Entrenar un modelo SVD
Se crea una instancia del modelo SVD con los parámetros especificados, como el número de factores latentes, el número de épocas, la tasa de aprendizaje y la regularización. El modelo se entrena utilizando el conjunto de entrenamiento.

### Paso 7: Guardar el modelo entrenado en un archivo
Se guarda el modelo entrenado en un archivo pkl utilizando la función `dump` de Surprise. El nombre del archivo se especifica como 'modelo_entrenado.pkl'.

Este código permite entrenar un modelo SVD en un conjunto de datos de películas y calificaciones, y guardar el modelo entrenado para su uso posterior.

optimizado con reduccion de tiempo y exportar modelo

In [3]:
from surprise import dump

# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd

# importar datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id', 'title'])
df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId', 'score', 'movieId'])
df_movie = df_movie.reset_index(drop=True)

# Crear un objeto Reader para analizar los datos de entrada
reader = Reader(rating_scale=(1.0, 5.0))

# Limitar los datos para evitar overfitting durante el aprendizaje
limite = 205000
data = Dataset.load_from_df(df_scores[['userId', 'movieId', 'score']][:limite], reader)

# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

# Entrenar un modelo SVD
n_factors = 30
n_epochs = 80
lr_all = 0.005
reg_all = 0.02
model = SVD(n_factors=n_factors, lr_all=lr_all, reg_all=reg_all, n_epochs=n_epochs)
model.fit(trainset)

# Guardar el modelo entrenado en un archivo
model_file = 'modelo_entrenado.pkl'
dump.dump(model_file, algo=model)


### Función `obtener_peliculas_recomendadas(user_id)`

Esta función utiliza un modelo entrenado previamente y un conjunto de datos de películas y calificaciones para recomendar películas para un usuario específico. Toma como entrada el `user_id` del usuario para el que se desean obtener las recomendaciones y devuelve una lista de títulos de películas recomendadas que el usuario aún no ha visto.

### Parámetros:
- `user_id` (int): El ID del usuario para el que se desean obtener las recomendaciones de películas.

### Retorno:
- `recommended_movies_titles` (list): Una lista de títulos de películas recomendadas para el usuario especificado.

### Pasos del proceso:
1. Cargar el modelo entrenado desde un archivo pkl previamente exportado.
2. Obtener las películas que el usuario ha visto previamente.
3. Generar una lista de películas no vistas por el usuario basada en las calificaciones existentes.
4. Utilizar el modelo cargado para hacer predicciones sobre las películas no vistas.
5. Ordenar las predicciones en orden descendente de las estimaciones de calificación.
6. Obtener los ID de las películas recomendadas.
7. Obtener los títulos de las películas recomendadas a partir del conjunto de datos de películas y calificaciones.
8. Filtrar las películas recomendadas para incluir solo aquellas que el usuario no ha visto previamente.
9. Devolver la lista de títulos de películas recomendadas para el usuario.

## Interfaz de Gradio para mostrar las películas recomendadas

La interfaz de Gradio permite mostrar la lista de películas recomendadas para un usuario específico. Toma el `user_id` como entrada y muestra los resultados en un campo de texto.

### Funcionamiento:
1. La interfaz solicita al usuario que ingrese el `user_id`.
2. La función `interfaz_peliculas_recomendadas` se ejecuta con el `user_id` proporcionado como entrada.
3. La función llama a la función `obtener_peliculas_recomendadas` para obtener la lista de 5 películas recomendadas para el usuario.
4. Si no hay películas recomendadas en ese momento o el usuario ya ha visto todas las películas recomendadas, se muestra un mensaje correspondiente.
5. Si hay películas recomendadas que el usuario no ha visto, se muestra la lista de títulos de películas recomendadas.

La interfaz de Gradio proporciona una manera interactiva para que los usuarios ingresen su ID y obtengan recomendaciones personalizadas de películas.

In [4]:
import gradio as gr
from surprise import dump
import pandas as pd

# Cargar el modelo desde el archivo pkl
model_file = 'modelo_entrenado.pkl'
model = dump.load(model_file)[1]

# Importar datos necesarios
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id', 'title'])
df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId', 'score', 'movieId'])
df_movie = df_movie.reset_index(drop=True)

def obtener_recomendaciones(user_id):
    # Obtener las películas y series ya vistas por el usuario
    movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
    
    # Obtener las películas no vistas por el usuario
    unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
    
    # Crear el conjunto de datos de prueba con las películas no vistas
    testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
    
    # Obtener las predicciones del modelo para las películas no vistas
    predictions = model.test(testset)
    
    # Obtener las películas recomendadas
    recommended_movies = [(pred.iid, pred.est) for pred in predictions]
    recommended_movies.sort(key=lambda x: x[1], reverse=True)
    recommended_movie_ids = [movie[0] for movie in recommended_movies]
    
    # Obtener los títulos de las películas recomendadas
    recommended_movies_titles = df_movie[df_movie['id'].isin(recommended_movie_ids)]['title'].tolist()
    
    # Obtener las primeras 5 películas recomendadas que el usuario no haya visto previamente
    recommended_movies_final = []
    for title in recommended_movies_titles:
        if len(recommended_movies_final) >= 5:
            break
        if title not in recommended_movies_final:
            recommended_movies_final.append(title)
    
    return recommended_movies_final

# Función para la interfaz de Gradio
def recommend_movies(user_id):
    recommended_movies = obtener_recomendaciones(user_id)
    return "\n".join(recommended_movies)

iface = gr.Interface(fn=recommend_movies, inputs="text", outputs="text", title="Recomendador de Películas")
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
